In [90]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# from langchain_community.llms import Ollama
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [177]:
# llm = HuggingFaceEndpoint(repo_id="google/gemma-2-2b-it",
#                           task = "text-generation")
# model = ChatHuggingFace(llm = llm)
# response = model.invoke("What is the capital of England?")
# print(response.content)

In [180]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    # model="gemma2-9b-it",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [92]:
response = model.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

print(response.content)

<think>
Alright, I need to figure out how to categorize each of these expenses appropriately. Let me go through each one step by step.

First up is "ISS Catering Services De Meern." ISS is known for providing catering and facility management services, so this would fall under "Food & Catering."

Next is "Vishandel Sier AMSTELVEEN." Vishandel Sier is a fish shop, so that should be categorized under "Groceries."

Then we have "Ministerie van Justitie en Veiligheid." That's the Dutch Ministry of Justice and Security, which is a government institution. So the category here should be "Government Services."

Moving on to "Etos AMSTERDAM NLD." Etos is a well-known pharmacy chain in the Netherlands, so this would fit under "Pharmacy."

Lastly, "Bistro Bar Amsterdam" sounds like a place where you'd go for dining or drinks, so the appropriate category would be "Dining."

I think that covers all the expenses with suitable categories. I should make sure each category is correctly assigned and then

### Read transaction data

In [11]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [12]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [14]:
unique_transactions[0:10]

array(['Belastingdienst', 'Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with LLM

In [124]:
categories_and_descriptions = [("Housing", "This includes your mortgage/rent, property taxes, homeowner insurance, and HOA fees"),
              ("Utilities", "Your gas, electricity, water, trash removal, and internet bills all fall under utilities"),
              ("Transportation", "Car payments, gas, public transportation, car insurance, parking costs"),
              ("Healthcare",  "Medical expenses, insurance premiums, copays, and prescription costs"),
              ("Food", "Groceries, dining out, coffee shop purchases"),
              ("Personal Care", "Personal hygiene items, haircuts, beauty products, gym memberships"),
              ("Debt Payments", "student loans, personal loans, etc."),
              ("Savings and Investments", "Effort to manage money for the future"),
              ("Shopping and Entertainment", "Clothing, electronics, entertainment, movies, concerts, subscription services, streaming"),
              ("Education and Professional Development", "Tuition, fees, supplies, books, and course materials, conferences, industry courses, software subscriptions, etc."),
              ("Taxes", "Includes all tax-related expenditures you’ve made (non-refundable promoters, etc.)"),
              ("Travel & Vacations", " Be generous with this if you're a travel enthusiast!")
              ]

categories = [cat for (cat, dec) in categories_and_descriptions]
descriptions = [dec for (cat, dec) in categories_and_descriptions]
comb = "\n".join([f"{i+1}. {cat}: {dec} \n" for (i, (cat, dec)) in enumerate(categories_and_descriptions)])

In [125]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from typing import Literal, Optional

class Transaction(BaseModel):
    category: Literal['Housing', 'Utilities','Transportation','Healthcare','Food',
                      'Personal Care','Debt Payments','Savings and Investments','Shopping and Entertainment',
                        'Education and Professional Development','Taxes','Travel & Vacations'] = Field(description="choose best category for the given bank transaction out of the following categories: " + "; ".join(categories))
    
parser = PydanticOutputParser(pydantic_object=Transaction)

In [144]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', 'You are a personal assistant whose job is to categorise given personal bank transactions by choosing the best fitting category from one of the following categories : \n\n' + comb),
    ('human', 'Transaction details: {query}; \n {format_instructions}')
])

In [171]:
def categorize_transactions(df):

    categories = []

    for i in range(len(df)):
        transaction = "; ".join([str(x) for x in list(df.iloc[i].values)])
        chain = prompt | model | parser
        categories.append(chain.invoke({'query': transaction, 'format_instructions': parser.get_format_instructions()}))
    
    categories = [x.category for x in categories]
    
    categories_df = df.copy()
    categories_df["category"] = categories
    
    return categories_df

In [156]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [181]:
categories_df = categorize_transactions(df)

In [182]:
categories_df.head()

,Date,Name / Description,Expense/Income,Amount (EUR),category
0,2023-12-30,Belastingdienst,Expense,9.96,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,Food
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,Housing
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,Food
4,2023-12-29,Selling Paintings,Income,13.63,Education and Professional Development


In [183]:
categories_df.category.unique()

array(['Taxes', 'Food', 'Housing',
       'Education and Professional Development',
       'Shopping and Entertainment', 'Healthcare', 'Transportation',
       'Personal Care', 'Savings and Investments'], dtype=object)

In [184]:
categories_df.category.value_counts()

category
Food                                      7
Shopping and Entertainment                6
Education and Professional Development    4
Personal Care                             4
Transportation                            2
Savings and Investments                   2
Taxes                                     1
Housing                                   1
Healthcare                                1
Name: count, dtype: int64

In [185]:
categories_df.to_csv("transactions_2022_2023_categorized.csv", index=False)